In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from joblib import dump
from joblib import load
import numpy as np

Read in [Honey Production Dataset](https://www.kaggle.com/jessicali9530/honey-production) from honeyproduction.csv

In [3]:
honey_data = pd.read_csv('honeyproduction.csv')

Use sklearn to run a simple regression model. A bias (intercept) term is included by default.<br>
X is a 2D numpy array of features where each row is a feature and each column is one data point (state-year)<br>
y is a 1D numpy array of responses (avg. honey per hive) to correspond with X

In [ ]:
model = LinearRegression(verbose=3).fit(X,y)

# save model object to a file
dump(model, 'model.joblib')

Now we use the trained model to generate some predictions. First, optionally load in the model from disk (if it wasn't just trained so it's not already in memory)

In [ ]:
model = load('model.joblib')

pred_X is a 2D numpy array of feature vectors that you would like to predict - each row should correspond to one feature and each column to one data point

In [ ]:
pred_X = numpy.array([]) # fill in with values to estimate

responses = model.predict(pred_X)